In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/train/000000000001.csv", sep='\t')

In [ ]:
data.shape

(116598, 82)

In [ ]:
data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
0,2541188,57,26325,22294,9018,25604,943,27941,19203,21533,...,2.855607,2.284486,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
1,2541440,60,5156,22294,18971,21545,3448,27941,19606,14659,...,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0
2,2541480,65,30256,22294,11104,21545,20366,27941,19203,31372,...,0.571121,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,1,0
3,2541780,63,17216,7152,15742,21545,23877,27941,19606,869,...,0.000000,0.000000,0.000000,0.347077,0.000000,0.0,0.0,0.0,0,0
4,2541833,60,9317,22294,26866,21545,32370,27941,21218,14659,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0


In [ ]:
data.isnull().sum()

,0
f_0,0
f_1,0
f_2,0
f_3,0
f_4,0
...,...
f_77,0
f_78,0
f_79,0
is_clicked,0


In [ ]:
data=data.dropna(axis=0)

In [ ]:
data['is_clicked'].value_counts()

,count
is_clicked,
0,41392
1,16499


In [ ]:
data['is_installed'].value_counts()

,count
is_installed,
0,44414
1,13477


In [ ]:
data.nunique()

,0
f_0,57891
f_1,22
f_2,126
f_3,5
f_4,576
...,...
f_77,3
f_78,7
f_79,4
is_clicked,2


In [ ]:
#training dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.utils import resample

X = data.drop(columns=['is_clicked', 'is_installed'])
y_clicked = data['is_clicked']

X_train, X_val, y_clicked_train, y_clicked_val = train_test_split(
    X, y_clicked, test_size=0.2, random_state=42
)



In [ ]:
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB


models = {
    "Decision Tree" : DecisionTreeClassifier(random_state=42),
    "Random Forest" : RandomForestClassifier(random_state=42),
    "Logistic Regression" : LogisticRegression(random_state=42),
    "XGBoost" : XGBClassifier(random_state=42),
    "Naive Bayes" : GaussianNB()
}
accuracy={}

for name, model in models.items():
  model.fit(X_train, y_clicked_train)
  y_pred = model.predict(X_val)
  accuracy_1 = accuracy_score(y_clicked_val, y_pred)

  print(f"{name} : {accuracy_1}")

Decision Tree : 0.7637101649537956
Random Forest : 0.8615597201830901
Logistic Regression : 0.7174194662751533
XGBoost : 0.865100613179031
Naive Bayes : 0.7146558424734433


In [ ]:
#resempled data
data_train = pd.concat([X_train, y_clicked_train], axis=1)

print(data_train['is_clicked'].value_counts())

click_0= data_train[data_train['is_clicked'] == 0]
click_1= data_train[data_train['is_clicked'] == 1]

upsamp=resample(click_1, replace = True, n_samples=len(click_0), random_state=42)
data_upsampled = pd.concat([upsamp,click_0])
print(data_upsampled['is_clicked'].value_counts())

X_train = data_upsampled.drop(columns=['is_clicked'])
y_clicked_train = data_upsampled['is_clicked']

is_clicked
0    33085
1    13227
Name: count, dtype: int64
is_clicked
1    33085
0    33085
Name: count, dtype: int64


In [ ]:
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB


models = {
    "Decision Tree" : DecisionTreeClassifier(random_state=42),
    "Random Forest" : RandomForestClassifier(random_state=42),
    "Logistic Regression" : LogisticRegression(random_state=42),
    "XGBoost" : XGBClassifier(random_state=42),
    "Naive Bayes" : GaussianNB()
}
accuracy= {}


for name, model in models.items():
  model.fit(X_train, y_clicked_train)
  y_pred = model.predict(X_val)
  accuracy_2 = accuracy_score(y_clicked_val, y_pred)

  print(f"{name} : {accuracy_2}")

Decision Tree : 0.7774419207185422
Random Forest : 0.8610415407202695
Logistic Regression : 0.5900336816650833
XGBoost : 0.8450643406166336
Naive Bayes : 0.7146558424734433


In [ ]:
#Support vector machine

from sklearn.svm import SVC
from sklearn import svm

clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_clicked_train)

#Predict the response for test dataset
y_pred = clf.predict(X_val)
print(accuracy_score(y_clicked_val, y_pred))

In [ ]:
clf_rbf = svm.SVC(kernel='rbf')
clf_rbf.fit(X_train, y_clicked_train)

#Predict the response for test dataset
y_pred = clf_rbf.predict(X_val)
print("rbf kenel: ", accuracy_score(y_clicked_val, y_pred))

In [ ]:
clf_sig = svm.SVC(kernel='sigmoid')
clf_sig.fit(X_train, y_clicked_train)

#Predict the response for test dataset
y_pred = clf_sig.predict(X_val)
print("sigmoid kenel: ", accuracy_score(y_clicked_val, y_pred))

In [ ]:
clf_poly = svm.SVC(kernel='poly')
clf_poly.fit(X_train, y_clicked_train)

#Predict the response for test dataset
y_pred = clf_poly.predict(X_val)
print("poly kenel: ", accuracy_score(y_clicked_val, y_pred))